In [14]:
# Import Dependencies
import os
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st
from scipy.stats import sem
from scipy.stats import linregress

In [16]:
# Set files input path, and read in pandas. We use the recently updated dataset on 06/29/2020.
global_dataset=pd.read_csv('../data_set/Global/06-29-2020.csv')
us_dataset=pd.read_csv('../data_set/US/06-29-2020.csv')
population_dataset=pd.read_csv('../data_set/Global Population/population.csv')
lockdown_level=pd.read_csv('../data_set/Lockdown data/global_countries_lockdown.csv')


In [3]:
# Display the dataset for US dataset to look for unnecessary values
us_dataset
# In this dataset, the Grand Princess and Diamond Princess will not be used, since they are cruise ships, 
# and not counted as the states.


,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,FIPS,Incident_Rate,People_Tested,People_Hospitalized,Mortality_Rate,UID,ISO3,Testing_Rate,Hospitalization_Rate
0,Alabama,US,2020-06-30 04:34:02,32.3182,-86.9023,37175,929,18866.0,17380.0,1,758.180652,393293.0,2725.0,2.498991,84000001.0,USA,8021.173992,7.330195
1,Alaska,US,2020-06-30 04:34:02,61.3707,-152.4044,901,14,525.0,362.0,2,123.163989,108709.0,NaN,1.553829,84000002.0,USA,14860.193153,NaN
2,American Samoa,US,2020-06-30 04:34:02,-14.2710,-170.1320,0,0,NaN,0.0,60,0.000000,696.0,NaN,NaN,16.0,ASM,1250.876152,NaN
3,Arizona,US,2020-06-30 04:34:02,33.7298,-111.4312,74545,1598,9179.0,63768.0,4,1024.150273,511423.0,4634.0,2.143672,84000004.0,USA,7026.279494,6.216379
4,Arkansas,US,2020-06-30 04:34:02,34.9697,-92.3731,20257,265,14066.0,5926.0,5,671.249690,302536.0,1380.0,1.308190,84000005.0,USA,10025.038074,6.812460
5,California,US,2020-06-30 04:34:02,36.1162,-119.6816,223646,5983,NaN,217663.0,6,566.017255,4061692.0,NaN,2.675210,84000006.0,USA,10279.583611,NaN
6,Colorado,US,2020-06-30 04:34:02,39.0598,-105.3111,32494,1681,4459.0,26354.0,8,564.255767,319565.0,5401.0,5.173263,84000008.0,USA,5549.221218,16.621530
7,Connecticut,US,2020-06-30 04:34:02,41.5978,-72.7554,46362,4320,8053.0,33989.0,9,1300.372172,444976.0,10268.0,9.317976,84000009.0,USA,12480.790467,22.147448
8,Delaware,US,2020-06-30 04:34:02,39.3185,-75.5071,11376,507,6665.0,4204.0,10,1168.250213,109931.0,NaN,4.456751,84000010.0,USA,11289.285700,NaN
9,Diamond Princess,US,2020-06-30 04:34:02,NaN,NaN,49,0,NaN,49.0,88888,NaN,NaN,NaN,0.000000,84088888.0,USA,NaN,NaN


In [4]:
# Get the indices of the rows that are not used
unused_rows=us_dataset.loc[(us_dataset['Province_State']=='Grand Princess')|(us_dataset['Province_State']=='Diamond Princess')]
unused_rows

,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,FIPS,Incident_Rate,People_Tested,People_Hospitalized,Mortality_Rate,UID,ISO3,Testing_Rate,Hospitalization_Rate
9,Diamond Princess,US,2020-06-30 04:34:02,NaN,NaN,49,0,NaN,49.0,88888,NaN,NaN,NaN,0.000000,84088888.0,USA,NaN,NaN
13,Grand Princess,US,2020-06-30 04:34:02,NaN,NaN,103,3,NaN,100.0,99999,NaN,NaN,NaN,2.912621,84099999.0,USA,NaN,NaN


In [20]:
# Drop the 9th and 13th rows and some extra columns what would not be used, such as: 
# Country_Region,Last_Update,FIPS,People_Tested,People_Hospitalized,Morality_rate, and last 4 columns.
cleaned_us_dataset=us_dataset.drop(us_dataset.index[[9,13]])

# Drop unused columns:
cleaned_us_dataset=cleaned_us_dataset[['Province_State','Last_Update','Lat','Long_','Confirmed','Deaths','Recovered','Active','Incident_Rate']]
cleaned_us_dataset=cleaned_us_dataset.reset_index()

# Delete the old index column
del cleaned_us_dataset['index']

# Fill the NaN cells with 0
cleaned_us_dataset=cleaned_us_dataset.fillna(0)

# Rename the Long_ column, so the df will have cleaner look
cleaned_us_dataset=cleaned_us_dataset.rename(columns={
    'Long_':'Lng'
})

# Display the cleaned up dataset for US
cleaned_us_dataset.head()

,Province_State,Last_Update,Lat,Lng,Confirmed,Deaths,Recovered,Active,Incident_Rate
0,Alabama,2020-06-30 04:34:02,32.3182,-86.9023,37175,929,18866.0,17380.0,758.180652
1,Alaska,2020-06-30 04:34:02,61.3707,-152.4044,901,14,525.0,362.0,123.163989
2,American Samoa,2020-06-30 04:34:02,-14.2710,-170.1320,0,0,0.0,0.0,0.000000
3,Arizona,2020-06-30 04:34:02,33.7298,-111.4312,74545,1598,9179.0,63768.0,1024.150273
4,Arkansas,2020-06-30 04:34:02,34.9697,-92.3731,20257,265,14066.0,5926.0,671.249690


In [21]:
# Display the Global dataset, also the population dataset, 
global_dataset.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
0,45001.0,Abbeville,South Carolina,US,2020-06-30 04:33:48,34.223334,-82.461707,106,0,0,106,"Abbeville, South Carolina, US",432.176785,0.000000
1,22001.0,Acadia,Louisiana,US,2020-06-30 04:33:48,30.295065,-92.414197,850,37,0,813,"Acadia, Louisiana, US",1369.973406,4.352941
2,51001.0,Accomack,Virginia,US,2020-06-30 04:33:48,37.767072,-75.632346,1039,14,0,1025,"Accomack, Virginia, US",3215.125634,1.347449
3,16001.0,Ada,Idaho,US,2020-06-30 04:33:48,43.452658,-116.241552,2040,23,0,2017,"Ada, Idaho, US",423.599474,1.127451
4,19001.0,Adair,Iowa,US,2020-06-30 04:33:48,41.330756,-94.471059,15,0,0,15,"Adair, Iowa, US",209.731544,0.000000


In [22]:
# Display the population_dataset
population_dataset.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Population
0,4,AF,AFG,4.0,NaN,NaN,NaN,Afghanistan,33.93911,67.709953,Afghanistan,38928341.0
1,8,AL,ALB,8.0,NaN,NaN,NaN,Albania,41.15330,20.168300,Albania,2877800.0
2,12,DZ,DZA,12.0,NaN,NaN,NaN,Algeria,28.03390,1.659600,Algeria,43851043.0
3,20,AD,AND,20.0,NaN,NaN,NaN,Andorra,42.50630,1.521800,Andorra,77265.0
4,24,AO,AGO,24.0,NaN,NaN,NaN,Angola,-11.20270,17.873900,Angola,32866268.0


In [23]:
# Find the US dataframe within the global_dataset, and count them to confirm the correct data.
# us_grouped=global_dataset.loc[global_dataset['Country_Region']=='US']
# len(us_grouped) = 3114
# So we found that there are 3114 rows for the US within the global_dataset, 
# so the rest would be 3785-3114 = 671 row entries for the global.
# global_dataset=global_dataset[global_dataset['Country_Region']!='US']
# len(global_dataset) = 671

# Drop some of the unecessary columns
global_dataset=global_dataset[['Province_State','Country_Region','Last_Update','Lat','Long_','Confirmed','Deaths','Recovered','Active','Incidence_Rate']]

# Rename some columns so the dataframe will look cleaner
cleaned_global_dataset=global_dataset.rename(columns={
    'Long_':'Lng',
    'Incidence_Rate':'Incident_Rate'
})
# Display the final global_dataset to look for null cells
cleaned_global_dataset

# Fill the Province_State NaN with unidentified as marker for map, since they have geocoordinate values
cleaned_global_dataset['Province_State']=cleaned_global_dataset['Province_State'].fillna(value='Unidentified')
# Fill other NaN (if any) with 0
cleaned_global_dataset=cleaned_global_dataset.fillna(0)
# Reset the index for df
cleaned_global_dataset.reset_index()
cleaned_global_dataset=cleaned_global_dataset.reset_index()
del cleaned_global_dataset['index']
# Display the df
cleaned_global_dataset.head()

,Province_State,Country_Region,Last_Update,Lat,Lng,Confirmed,Deaths,Recovered,Active,Incident_Rate
0,South Carolina,US,2020-06-30 04:33:48,34.223334,-82.461707,106,0,0,106,432.176785
1,Louisiana,US,2020-06-30 04:33:48,30.295065,-92.414197,850,37,0,813,1369.973406
2,Virginia,US,2020-06-30 04:33:48,37.767072,-75.632346,1039,14,0,1025,3215.125634
3,Idaho,US,2020-06-30 04:33:48,43.452658,-116.241552,2040,23,0,2017,423.599474
4,Iowa,US,2020-06-30 04:33:48,41.330756,-94.471059,15,0,0,15,209.731544


In [9]:
# Work with the global population_dataset to get the population for the us and the rest.
population_df=pd.DataFrame(columns=['Country_Region','Population'])
# Get the list of names countries in the global.
global_list_name=cleaned_global_dataset['Country_Region'].unique()
# grouped_df=population_dataset.groupby(['Country_Region'])
# sum1=(grouped_df['Population'].sum())/2
    
# population_df['Population']=sum1        
# population_df.reset_index()
# population_df.loc[population_df['Country_Region']=='US']
for i,row in population_dataset.iterrows():
    if row['Country_Region'] == row['Combined_Key']:
        population_df=population_df.append({'Country_Region':row['Country_Region'],'Population':row['Population']},ignore_index=True)
        

population_df.sort_values(by='Country_Region')


,Country_Region,Population
0,Afghanistan,38928341.0
1,Albania,2877800.0
2,Algeria,43851043.0
3,Andorra,77265.0
4,Angola,32866268.0
...,...,...
179,West Bank and Gaza,5101416.0
180,Western Sahara,597330.0
181,Yemen,29825968.0
182,Zambia,18383956.0


In [10]:
grouped_global=cleaned_global_dataset.groupby(['Country_Region','Province_State'])
mortality_rate=grouped_global['Deaths'].sum().reset_index()
mortality_rate.sort_values(by='Country_Region')
# mortality_rate.loc[mortality_rate['Country_Region']=='US']
# sorted_value=mortality_rate.sort_values(by='Country_Region')
# population_df['Mortality']=sorted_value['Deaths']
# # # population_df.loc[:,'Mortality'=mortality_rate]
# joined_df=population_df.join(mortality_rate,how='left',on='Country_Region')


us_mortality=mortality_rate.loc[mortality_rate['Country_Region']=='US']
us_mortality

,Country_Region,Province_State,Deaths
618,US,Alabama,929
619,US,Alaska,14
620,US,Arizona,1598
621,US,Arkansas,265
622,US,California,5983
623,US,Colorado,1681
624,US,Connecticut,4320
625,US,Delaware,507
626,US,Diamond Princess,0
627,US,District of Columbia,551


In [11]:
lockdown_df=pd.DataFrame(lockdown_level)
droped_lockdown_df=lockdown_df.drop(lockdown_df.index[[0,-1]])
droped_lockdown_df=droped_lockdown_df.reset_index()
del droped_lockdown_df['index']
droped_lockdown_df.rename(columns={
    "vteCOVID-19 pandemic lockdowns":'Countries_Territories',
    'vteCOVID-19 pandemic lockdowns.1':'Place',
    'vteCOVID-19 pandemic lockdowns.2':'Start_date',
    'vteCOVID-19 pandemic lockdowns.3':'End_date',
    'vteCOVID-19 pandemic lockdowns.4':'Level'
    
},inplace=True)


national_lockdown_df=droped_lockdown_df.loc[droped_lockdown_df['Level']=='National']
regional_lockdown_df=droped_lockdown_df.loc[droped_lockdown_df['Countries_Territories']== 'United States']

national_lockdown_df.fillna(0)
national_lockdown_df=national_lockdown_df.reset_index()
del national_lockdown_df['index']
national_lockdown_df.head(50)

,Countries_Territories,Place,Start_date,End_date,Level
0,Albania,Albania,2020-03-13[3],2020-06-01[4],National
1,Argentina,Argentina,2020-03-19[7],2020-06-28[8],National
2,Armenia,Armenia,2020-03-24[9],2020-05-04[10],National
3,Australia,Australia,2020-03-23[11],NaN,National
4,Austria,Austria,2020-03-16[12],2020-04-13[13],National
5,Azerbaijan,Azerbaijan,2020-03-31[14],2020-04-20[14],National
6,Bangladesh,Bangladesh,2020-03-26[15],2020-05-16[16],National
7,Barbados,Barbados,2020-03-28[17],2020-05-03[17],National
8,Belgium,Belgium,2020-03-18[18],2020-05-04 [19],National
9,Bermuda,Bermuda,2020-04-04[20],2020-05-02[21],National


In [12]:
regional_lockdown_df.fillna(0)

,Countries_Territories,Place,Start_date,End_date,Level
100,United States,California,2020-03-19[152],0,State
101,United States,Clark County in Nevada,2020-03-20,0,County
102,United States,Connecticut,2020-03-23[153],2020-04-22[154],State
103,United States,Illinois,2020-03-21[155],2020-05-30[156],State
104,United States,Kansas City in Kansas,2020-03-24[157],2020-04-19[158],City
105,United States,Massachusetts,2020-03-24[159],2020-05-04[159],State
106,United States,Michigan,2020-03-24[160],2020-04-13[154],State
107,United States,New York,2020-03-22[161],2020-06-13[162],State
108,United States,Oregon,2020-03-24[163],0,State
109,United States,Wisconsin,2020-03-24[164],0,State
